In [ ]:
import os
import cv2
import time
import pandas as pd
import numpy as np
from PIL import Image
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

dataset_path = r"S:\ShyamKrishna_Chunduri\UofM\UofMSem4\IntroToAI\project\dataset"


In [ ]:
# Define the categories
categories = ['incorrect_mask\mc', 'incorrect_mask\mmc',
              'without_mask\complex', 'without_mask\simple',
              'with_mask\complex', 'with_mask\simple']

# Function to plot images
def plot_images(image_dir, category, num_images=4):
    fig, axes = plt.subplots(1, num_images, figsize=(5, 5))
    fig.suptitle(category, fontsize=5)
    
    for i, file_name in enumerate(os.listdir(image_dir)[:num_images]):
        img_path = os.path.join(image_dir, file_name)
        img = plt.imread(img_path)
        axes[i].imshow(img)
        axes[i].axis('off')
    plt.show()

# Plot a few images from each category
for category in categories:
    image_dir = os.path.join(dataset_path, category)
    plot_images(image_dir, category)

In [ ]:
# Initialize ImageDataGenerator for training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)  # Use 20% of the data for validation

# Prepare the training and validation generators
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training')  # Set as training data

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # Set as validation data

In [ ]:
# Accessing class_indices attribute
class_indices = validation_generator.class_indices

# Getting class names from the class_indices dictionary
class_names = list(class_indices.keys())

# Printing class names
print("Class names in the training generator:")
for name in class_names:
    print(name)

In [ ]:
# Get a batch of images and labels from the train_generator
images, labels = next(train_generator)

# Plot the first image in the batch
plt.imshow(images[0])
plt.title("Sample Image from Train Generator")
plt.axis('off')  # Turn off axis labels and ticks
plt.show()

# If you also want to display the label for this image
print("Label for this image:", labels[0])


# Get a batch of images and labels from the train_generator
images, labels = next(validation_generator)

# Plot the first image in the batch
plt.imshow(images[0])
plt.title("Sample Image from Validation Generator")
plt.axis('off')  # Turn off axis labels and ticks
plt.show()

# If you also want to display the label for this image
print("Label for this image:", labels[0])

In [ ]:
# Define the CNN model
model = Sequential([
    tensorflow.keras.Input(shape=(256, 256, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming three classes: with_mask, without_mask, incorrect_mask
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_generator.n

In [ ]:
# Two Pointer Approach

ans = []
 
 
def divisor(val):
     result = []
    
     for i in range(1, val + 1):
         ans.append(i)
    
     i = 0
     j = len(ans) - 1
    
     while i < j:
    
         if ans[i] * ans[j] == ans[-1]:
             result.append(ans[i])
             result.append(ans[j])
             i += 1
         else:
             j -= 1
    
     return sorted(result)
 
     
print(divisor(11630))

In [ ]:
#Configure early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Configure model checkpointing
model_checkpoint = ModelCheckpoint(
    'best_mask_detection_model.keras',
    save_best_only=True,
    monitor='val_loss',
    mode='min'
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[early_stopping, model_checkpoint]  # Add model_checkpoint here
)

# Save the final model
model.save('mask_detection_model_final.keras')